# Autoencoder Nasa Randomized dataset

In [ ]:
import numpy as np
import pandas as pd
import scipy.io
import math
import os
import ntpath
import sys
import logging
import time
import sys
import random

from importlib import reload
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Model


IS_COLAB = False
IS_TRAINING = True
RESULT_NAME = ""

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = "/content/drive/My Drive/CEM-Data-Experiment/cem-data-experiment/"
else:
    data_path = "../../"

sys.path.append(data_path)
from data_processing.nasa_random_data import NasaRandomizedData
from data_processing.prepare_rul_data import RulHandler

### Config logging

In [ ]:
reload(logging)
logging.basicConfig(format='%(asctime)s [%(levelname)s]: %(message)s', level=logging.DEBUG, datefmt='%Y/%m/%d %H:%M:%S')

# Load Data

In [ ]:
train_names_10k = [
        'RW_Skewed_Low_Room_Temp_DataSet_2Post/data/Matlab/RW13',
        'RW_Skewed_Low_Room_Temp_DataSet_2Post/data/Matlab/RW14',
        'RW_Skewed_Low_Room_Temp_DataSet_2Post/data/Matlab/RW15',

        'RW_Skewed_Low_40C_DataSet_2Post/data/Matlab/RW21',
        'RW_Skewed_Low_40C_DataSet_2Post/data/Matlab/RW22',

        'RW_Skewed_High_40C_DataSet_2Post/data/Matlab/RW25',
        'RW_Skewed_High_40C_DataSet_2Post/data/Matlab/RW26',
        'RW_Skewed_High_40C_DataSet_2Post/data/Matlab/RW27',
]

val_names_10k = [
        'RW_Skewed_Low_40C_DataSet_2Post/data/Matlab/RW23',
]

test_names_10k = [
        'RW_Skewed_Low_Room_Temp_DataSet_2Post/data/Matlab/RW16',
        'RW_Skewed_Low_40C_DataSet_2Post/data/Matlab/RW24',
        'RW_Skewed_High_40C_DataSet_2Post/data/Matlab/RW28',
]

train_names_20k = [
        'Battery_Uniform_Distribution_Variable_Charge_Room_Temp_DataSet_2Post/data/Matlab/RW1',
        'Battery_Uniform_Distribution_Variable_Charge_Room_Temp_DataSet_2Post/data/Matlab/RW2',
        'Battery_Uniform_Distribution_Variable_Charge_Room_Temp_DataSet_2Post/data/Matlab/RW7',

        #'Battery_Uniform_Distribution_Discharge_Room_Temp_DataSet_2Post/data/Matlab/RW3',
        'Battery_Uniform_Distribution_Discharge_Room_Temp_DataSet_2Post/data/Matlab/RW4',
        'Battery_Uniform_Distribution_Discharge_Room_Temp_DataSet_2Post/data/Matlab/RW5',

        #'Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/Matlab/RW9',
        #'Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/Matlab/RW10',
        #'Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/Matlab/RW11',
]

test_names_20k = [
        'Battery_Uniform_Distribution_Variable_Charge_Room_Temp_DataSet_2Post/data/Matlab/RW8',
        'Battery_Uniform_Distribution_Discharge_Room_Temp_DataSet_2Post/data/Matlab/RW6',
        #'Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/Matlab/RW12',
]

train_names_100k = [
        'RW_Skewed_High_Room_Temp_DataSet_2Post/data/Matlab/RW17',
        'RW_Skewed_High_Room_Temp_DataSet_2Post/data/Matlab/RW18',
        'RW_Skewed_High_Room_Temp_DataSet_2Post/data/Matlab/RW19',
]

test_names_no_100k = [
        'RW_Skewed_High_Room_Temp_DataSet_2Post/data/Matlab/RW20',
]

In [ ]:
nasa_data_handler = NasaRandomizedData(data_path)
rul_handler = RulHandler()

## Data preparation

In [ ]:
(train_x_10k, _, test_x_10k, _, battery_range_cycle_train_10k, battery_range_cycle_test_10k, _, _, _, _,
  val_x_10k, _, _, _, _) = nasa_data_handler.get_discharge_whole_cycle_future(train_names_10k, test_names_10k, val_names_10k)
(train_x_20k, _, test_x_20k, _, _, _, _, _, _, _) = nasa_data_handler.get_discharge_whole_cycle_future(train_names_20k, test_names_20k)
(train_x_100k, _, test_x_no_100k, _, _, _, _, _, _, _) = nasa_data_handler.get_discharge_whole_cycle_future(train_names_100k, test_names_no_100k)


train_x_20k = train_x_20k[:, ::2, :]
test_x_20k = test_x_20k[:, ::2, :]
train_x_100k = train_x_100k[:, ::10, :]
max_lenght = max(train_x_10k.shape[1], test_x_10k.shape[1], train_x_20k.shape[1], test_x_20k.shape[1], train_x_100k.shape[1], test_x_no_100k.shape[1])

train_x = np.zeros((
        train_x_10k.shape[0] + train_x_20k.shape[0] + train_x_100k.shape[0],
        max_lenght,
        train_x_10k.shape[2]))
train_x[:train_x_10k.shape[0], :train_x_10k.shape[1], :] = train_x_10k
train_x[train_x_10k.shape[0]:train_x_10k.shape[0]+train_x_20k.shape[0], :train_x_20k.shape[1], :] = train_x_20k
train_x[train_x_10k.shape[0]+train_x_20k.shape[0]:, :train_x_100k.shape[1], :] = train_x_100k

val_x = np.zeros((
        val_x_10k.shape[0],
        max_lenght,
        val_x_10k.shape[2]))
val_x[:val_x_10k.shape[0], :val_x_10k.shape[1], :] = val_x_10k

test_x = np.zeros((
        test_x_10k.shape[0] + test_x_20k.shape[0] + test_x_no_100k.shape[0],
        max_lenght,
        test_x_10k.shape[2]))
test_x[:test_x_10k.shape[0], :test_x_10k.shape[1], :] = test_x_10k
test_x[test_x_10k.shape[0]:test_x_10k.shape[0]+test_x_20k.shape[0], :test_x_20k.shape[1], :] = test_x_20k
test_x[test_x_10k.shape[0]+test_x_20k.shape[0]:, :test_x_no_100k.shape[1], :] = test_x_no_100k

print("train shape {}".format(train_x.shape))
print("val shape {}".format(val_x.shape))
print("test shape {}".format(test_x.shape))

train_x = train_x[:,:11800,:]
val_x = val_x[:,:11800,:]
test_x = test_x[:,:11800,:]
print("cut train shape {}".format(train_x.shape))
print("cut val shape {}".format(val_x.shape))
print("cut test shape {}".format(test_x.shape))

x_norm = rul_handler.Normalization()
x_norm.fit(train_x)
train_x = x_norm.normalize(train_x)
val_x = x_norm.normalize(val_x)
test_x = x_norm.normalize(test_x)

# Model training

In [ ]:
if IS_TRAINING:
    EXPERIMENT = "autoencoder_gl_nasa_randomized"

    experiment_name = time.strftime("%Y-%m-%d-%H-%M-%S") + '_' + EXPERIMENT
    print(experiment_name)

# Model definition

opt = tf.keras.optimizers.Adam(learning_rate=0.0002)
LATENT_DIM = 15

class Autoencoder(Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim

        encoder_inputs = layers.Input(shape=(train_x.shape[1], train_x.shape[2]))
        encoder_conv1 = layers.Conv1D(filters=16, kernel_size=10, strides=5, activation='relu', padding='same')(encoder_inputs)
        encoder_pool1 = layers.MaxPooling1D(5, padding='same')(encoder_conv1)
        encoder_conv2 = layers.Conv1D(filters=16, kernel_size=4, strides=2, activation='relu', padding='same')(encoder_pool1)
        encoder_pool2 = layers.MaxPooling1D(4, padding='same')(encoder_conv2)
        encoder_flat1 = layers.Flatten()(encoder_pool1)
        encoder_flat2 = layers.Flatten()(encoder_pool2)
        encoder_concat = layers.concatenate([encoder_flat1, encoder_flat2])
        encoder_outputs = layers.Dense(self.latent_dim, activation='relu')(encoder_concat)
        self.encoder = Model(inputs=encoder_inputs, outputs=encoder_outputs)

        decoder_inputs = layers.Input(shape=(self.latent_dim,))
        decoder_dense1 = layers.Dense(59*16, activation='relu')(decoder_inputs)
        decoder_reshape1 = layers.Reshape((59, 16))(decoder_dense1)
        decoder_upsample1 = layers.UpSampling1D(4)(decoder_reshape1)
        decoder_convT1 = layers.Conv1DTranspose(filters=16, kernel_size=4, strides=2, activation='relu', padding='same')(decoder_upsample1)
        decoder_upsample2 = layers.UpSampling1D(5)(decoder_convT1)
        decoder_convT2 = layers.Conv1DTranspose(filters=16, kernel_size=10, strides=5, activation='relu', padding='same')(decoder_upsample2)
        decoder_outputs = layers.Conv1D(3, kernel_size=3, activation='relu', padding='same')(decoder_convT2)
        self.decoder = Model(inputs=decoder_inputs, outputs=decoder_outputs)



    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(LATENT_DIM)
autoencoder.compile(optimizer=opt, loss='mse', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
autoencoder.encoder.summary()
autoencoder.decoder.summary()

In [ ]:
if IS_TRAINING:
    history = autoencoder.fit(train_x, train_x,
                                epochs=500, 
                                batch_size=32, 
                                verbose=1,
                                validation_data=(val_x, val_x)
                               )

In [ ]:
if IS_TRAINING:
    autoencoder.save_weights(data_path + 'results/trained_model/%s/model' % experiment_name)

    hist_df = pd.DataFrame(history.history)
    hist_csv_file = data_path + 'results/trained_model/%s/history.csv' % experiment_name
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)
    history = history.history

In [ ]:
if not IS_TRAINING:
    history = pd.read_csv(data_path + 'results/trained_model/%s/history.csv' % RESULT_NAME)
    autoencoder.load_weights(data_path + 'results/trained_model/%s/model' % RESULT_NAME)
    autoencoder.encoder.summary()
    autoencoder.decoder.summary()

In [ ]:
if not IS_TRAINING:
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(history)

### Testing

In [ ]:
results = autoencoder.evaluate(test_x, test_x, return_dict = True)
print(results)
max_rmse = 0
for index in range(test_x.shape[0]):
    result = autoencoder.evaluate(np.array([test_x[index, :, :]]), np.array([test_x[index, :, :]]), return_dict = True, verbose=0)
    max_rmse = max(max_rmse, result['rmse'])
print("Max rmse: {}".format(max_rmse))

# Results Visualization

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history['loss'],
                    mode='lines', name='train'))
if 'val_loss' in history:
    fig.add_trace(go.Scatter(y=history['val_loss'],
                    mode='lines', name='validation'))
fig.update_layout(title='Loss trend',
                  xaxis_title='epoch',
                  yaxis_title='loss',
                  width=1400,
                  height=600)
fig.show()

In [ ]:
train_predictions = autoencoder.predict(train_x)
labels = ['Voltage', 'Current', 'Temperature']

In [ ]:
for i in range(train_x.shape[2]):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=train_predictions[5,:,i],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=train_x[5,:,i],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on training - battery new',
                    xaxis_title='Step',
                    yaxis_title=labels[i],
                    width=1400,
                    height=600)
    fig.show()

In [ ]:
for i in range(train_x.shape[2]):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=train_predictions[int(battery_range_cycle_train_10k[0]/2),:,i],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=train_x[int(battery_range_cycle_train_10k[0]/2),:,i],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on training - middle life',
                    xaxis_title='Step',
                    yaxis_title=labels[i],
                    width=1400,
                    height=600)
    fig.show()

In [ ]:
for i in range(train_x.shape[2]):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=train_predictions[battery_range_cycle_train_10k[0]-5,:,i],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=train_x[battery_range_cycle_train_10k[0]-5,:,i],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on training - end of life',
                    xaxis_title='Step',
                    yaxis_title=labels[i],
                    width=1400,
                    height=600)
    fig.show()

In [ ]:
test_predictions = autoencoder.predict(test_x)
labels = ['Voltage', 'Current', 'Temperature']

In [ ]:
for i in range(train_x.shape[2]):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=test_predictions[0,:,i],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=test_x[0,:,i],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on testing - battery new',
                    xaxis_title='Step',
                    yaxis_title=labels[i],
                    width=1400,
                    height=600)
    fig.show()

In [ ]:
for i in range(train_x.shape[2]):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=test_predictions[int(battery_range_cycle_test_10k[0]/2),:,i],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=test_x[int(battery_range_cycle_test_10k[0]/2),:,i],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on testing - middle life',
                    xaxis_title='Step',
                    yaxis_title=labels[i],
                    width=1400,
                    height=600)
    fig.show()

In [ ]:
for i in range(train_x.shape[2]):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=test_predictions[battery_range_cycle_test_10k[0]-5,:,i],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=test_x[battery_range_cycle_test_10k[0]-5,:,i],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on testing - end of life',
                    xaxis_title='Step',
                    yaxis_title=labels[i],
                    width=1400,
                    height=600)
    fig.show()